# Project Management with the Github API 
## Primary Goal: Create new organization level project cards for all existing issues.
- Open issues will be placed in the `Triage` column
- Closed issues will have cards loaded in the `Deployed` column

Before you start:
You must have a github token and collaboration rights to the projects and repositories you're trying to access.

In [1]:
#load modules
import requests
import os
from pprint import pprint
import config #Where I save my Github token
import pandas as pd
import json

## Let's start by getting a list of all the open issues in the repository of interest.

In [86]:
# Define the input parameters
token = config.GITHUB_TOKEN
acc = "application/vnd.github.inertia-preview+json" #this is the Accept type you must have for working with Projects in the current Github Api
params = {"content_type" : "Issue", "state":"open"} #notice the content type is for issues and the issue state is open
HEADERS = {'Accept': f'{acc}','Authorization': f'token {token}'}
OWNER = "Thinkful-Ed"
REPO = "technical-project-management-programs"

In [87]:
# get all the open issues from a specific repository in a dictionary

ISSUES_DICT = {}
def get_issues_by_repository(repository):
    url = "https://api.github.com/repos/{}/{}/issues".format(OWNER, repository)
    headers = HEADERS
    r = requests.get(
        url=url,
        headers=headers, 
        params=params

    )
    issues = r.json()
    while "next" in r.links:
        print("\tNext Page: {}".format(r.links["next"]["url"]))
        r = requests.get(
            url=r.links["next"]["url"],
            headers=headers,
            params=params
        )
        issues.extend(r.json())
    ISSUES_DICT.update({repository: issues})
    return issues

issue_dict = get_issues_by_repository(REPO)

	Next Page: https://api.github.com/repositories/255397493/issues?content_type=Issue&state=open&page=2
	Next Page: https://api.github.com/repositories/255397493/issues?content_type=Issue&state=open&page=3


In [88]:
## Let's see how many open issues we have
len(issue_dict)

85

In [7]:
## Create a pandas dataframe from the open issues 
df = pd.DataFrame.from_dict(issue_dict)
df.columns

Index(['url', 'repository_url', 'labels_url', 'comments_url', 'events_url',
       'html_url', 'id', 'node_id', 'number', 'title', 'user', 'labels',
       'state', 'locked', 'assignee', 'assignees', 'milestone', 'comments',
       'created_at', 'updated_at', 'closed_at', 'author_association',
       'active_lock_reason', 'body', 'performed_via_github_app',
       'pull_request'],
      dtype='object')

This dataframe has loads of useful information about the issues submitted. We can investigate that further in a bit. Let's confirm we only have issues that are still open in our data by checking the value counts of 'state'.

In [8]:
df.state.value_counts()

open    85
Name: state, dtype: int64

Looks all good. We now need to shift over to the project. In this particular case the project is at the organization level rather than within the repository where the issues are located. * If this is the case in your project before proceeding make sure to link the repository of the issues to the project.*

In [41]:
#lets collect some info about our projects
#List all the projects in our organization
query_url = f"https://api.github.com/orgs/{OWNER}/projects"
r = requests.get(query_url, headers=HEADERS, params=params)
obj = r.json()
print(type(obj))

<class 'list'>
<class 'list'>


In [62]:
#print project names and id's
project_names = [item['name'] for item in obj]
project_ids = [item['id'] for item in obj]
for i in range(len(project_names)):
    print(project_names[i] + '\t ' + str(project_ids[i]))

Web Dev Curriculum Maintenance	 1519578
Design Curriculum Maintenance	 2511356
Data Science Curriculum Maintenance	 2511361
Data Analytics Curriculum Maintenance	 2511363
Thinkful Engineering Curriculum Maintenance	 2568836
Web Dev Instructor Feedback	 3052319
Product Management Maintenance	 3798803
Technical Project Management Maintenance	 5517614


Great, now we can look at all the metadata about our projects. Specifically we want to find the project id for the project we are interested in adding issue cards to.

In [71]:
#update our url with the specific project id and take a look at the cards in that project
proj_id = '5517614'
proj_url =f'https://api.github.com/projects/{proj_id}/columns'
r = requests.get(proj_url, headers=HEADERS, params=params)
obj = r.json()
card_names = [item['name'] for item in obj]
card_names

['Triage',
 'Backlog',
 '🔥🔥🔥🔥🔥',
 'In progress',
 'Instructional Designer',
 'High priority for SMEs',
 'Deployed']

In [78]:
#let's use the name to find the Triage column id
card_id = [item['id'] for item in obj if item['name'] == 'Triage'][0]
card_id

10926017

## Using the API to create new project cards
The key parameters here are setting the "content_id" to the issue.id and the "content_type" to "Issues". For creating new project cards for pull requests then change the "content_id" to pullrequest.id and "content_type" to "PullRequests".

In [94]:
#write a function which ingests issues and create a card in the Triage column
def create_card(issue_id):
    payload = {
        "content_id": issue_id,
        "content_type": "Issue",
    }
    response = requests.post(f'https://api.github.com/projects/columns/{card_id}/cards', headers=HEADERS, data=json.dumps(payload))
    print(response.json())
    if response.status_code != 200:
        response.raise_for_status()

In [83]:
#create a list of the issue id's from the issue dictionary
issue_list2 = [item['id'] for item in issue_dict]
issue_list2[:3]

[708461457, 707613667, 707457515]

In [112]:
#use a for loop to feed in all the issue id's from the list
for i in issue_list2:
    create_card(i)

events', 'type': 'User', 'site_admin': False}, 'created_at': '2020-09-25T19:05:25Z', 'updated_at': '2020-09-25T19:05:25Z', 'column_url': 'https://api.github.com/projects/columns/10926017', 'content_url': 'https://api.github.com/repos/Thinkful-Ed/technical-project-management-programs/issues/27'}
{'url': 'https://api.github.com/projects/columns/cards/46198398', 'project_url': 'https://api.github.com/projects/5517614', 'id': 46198398, 'node_id': 'MDExOlByb2plY3RDYXJkNDYxOTgzOTg=', 'note': None, 'archived': False, 'creator': {'login': 'AVJdataminer', 'id': 30507129, 'node_id': 'MDQ6VXNlcjMwNTA3MTI5', 'avatar_url': 'https://avatars0.githubusercontent.com/u/30507129?u=32df99b5d9bd9812ff17d3b295a682b5d840bcc2&v=4', 'gravatar_id': '', 'url': 'https://api.github.com/users/AVJdataminer', 'html_url': 'https://github.com/AVJdataminer', 'followers_url': 'https://api.github.com/users/AVJdataminer/followers', 'following_url': 'https://api.github.com/users/AVJdataminer/following{/other_user}', 'gists_

## Create a list of of closed tickets to add to the deployed column in our project


In [91]:
#update params to include only closed issues
params = {"content_type" : "Issue", "state":"closed"}
closed_issue_dict = get_issues_by_repository(REPO)
print(len(closed_issue_dict)) #check the number of closed issues
issue_list = [item['id'] for item in closed_issue_dict] #create a list of closed issue id's

	Next Page: https://api.github.com/repositories/255397493/issues?content_type=Issue&state=closed&page=2
41


In [92]:
#get the column id for the deployed column in the same project
#let's use the name to find the Triage column id
card_id = [item['id'] for item in obj if item['name'] == 'Deployed'][0]
card_id

10926023

In [95]:
#Loop over the closed issue id list with the create card function
for i in issue_list:
    create_card(i)

_events', 'type': 'User', 'site_admin': False}, 'created_at': '2020-09-25T22:54:28Z', 'updated_at': '2020-09-25T22:54:28Z', 'column_url': 'https://api.github.com/projects/columns/10926023', 'content_url': 'https://api.github.com/repos/Thinkful-Ed/technical-project-management-programs/issues/29'}
{'url': 'https://api.github.com/projects/columns/cards/46207829', 'project_url': 'https://api.github.com/projects/5517614', 'id': 46207829, 'node_id': 'MDExOlByb2plY3RDYXJkNDYyMDc4Mjk=', 'note': None, 'archived': False, 'creator': {'login': 'AVJdataminer', 'id': 30507129, 'node_id': 'MDQ6VXNlcjMwNTA3MTI5', 'avatar_url': 'https://avatars0.githubusercontent.com/u/30507129?u=32df99b5d9bd9812ff17d3b295a682b5d840bcc2&v=4', 'gravatar_id': '', 'url': 'https://api.github.com/users/AVJdataminer', 'html_url': 'https://github.com/AVJdataminer', 'followers_url': 'https://api.github.com/users/AVJdataminer/followers', 'following_url': 'https://api.github.com/users/AVJdataminer/following{/other_user}', 'gists